# OpenBook ai

Largest LM model

# Score Analysis

Importing sentence Transformer

In [3]:
!pip install sentence-transformers

Importing textstat

In [4]:
!pip install textstat

   ---------------------------------------- 0.0/105.1 kB ? eta -:--:--
   --- ------------------------------------ 10.2/105.1 kB ? eta -:--:--
   --------------- ----------------------- 41.0/105.1 kB 393.8 kB/s eta 0:00:01
   -------------------------- ------------ 71.7/105.1 kB 491.5 kB/s eta 0:00:01
   -------------------------------------- 105.1/105.1 kB 551.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 653.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.1 MB 787.7 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 804.6 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.1 MB 876.1 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.1 MB 888.4 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.1 MB 927.4 kB/s eta 0:00:02
   ----- --

Importing necessary Library

In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer, util
from textstat import textstat

In [72]:
def text_call(list):
  texts =list
    lengths = [len(text.split()) for text in texts]
    readability_scores = [textstat.flesch_reading_ease(text) for text in texts]
    length_and_complexity_scores = [length + readability_score for length, readability_score in zip(lengths, readability_scores)]
    
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    tfidf_scores = np.array(X.sum(axis=1)).flatten()
    
   
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts)
    embedding_magnitudes = [np.linalg.norm(embedding) for embedding in embeddings]
    
   
    count_vectorizer = CountVectorizer(stop_words='english')
    X_count = count_vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=3, random_state=42)
    lda.fit(X_count)
    
    
    text_topic_dist = lda.transform(X_count)
    
    
    topic_word_count = lda.components_.sum(axis=1)  
    text_topic_count = np.sum(text_topic_dist * topic_word_count, axis=1)
    
   
    combined_scores = pd.DataFrame({
        'Text': texts,
        'Length and Complexity Score': length_and_complexity_scores,
        'TF-IDF Score': tfidf_scores,
        'Embedding Magnitude Score': embedding_magnitudes,
        'Topic Coverage Score': text_topic_count
    })
    
    
    weights = {
        'Length and Complexity Score': 0.25,
        'TF-IDF Score': 0.25,
        'Embedding Magnitude Score': 0.25,
        'Topic Coverage Score': 0.25
    }
    combined_scores['Total Score'] = (
        combined_scores['Length and Complexity Score'] * weights['Length and Complexity Score'] +
        combined_scores['TF-IDF Score'] * weights['TF-IDF Score'] +
        combined_scores['Embedding Magnitude Score'] * weights['Embedding Magnitude Score'] +
        combined_scores['Topic Coverage Score'] * weights['Topic Coverage Score']
    )
    
    
    sorted_scores = combined_scores.sort_values(by='Total Score', ascending=False)
    print("\nScores for each text:")
    print(sorted_scores)
    
   
    highest_score_text = sorted_scores.iloc[0]
    print("\nText with the highest score:")
    print(f"Text: {highest_score_text['Text']}")
    print(f"Total Score: {highest_score_text['Total Score']}")
    passing_data = highest_score_text['Text']
    print(passing_data)
    return summary_call(passing_data)


# TEXT SUMMARIZATION

Importing bert

In [12]:
!pip install bert-extractive-summarizer

Importing neccessary module

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from summarizer import Summarizer

C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Load the BERTSUM model and tokenizer

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = Summarizer()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abina\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abina\.cache\huggingface\hub\models--bert-large-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generate summary

In [31]:
def summary_call(passing_data):
    summary = model(passing_data, ratio=0.5)
    return summary

In [16]:
print(summary)

The OSI (Open Systems Interconnection) model is a conceptual framework used to describe how data communication occurs between devices in a network. It consists of seven layers, each with specific functions:  1. Physical Layer: Deals with the physical transmission of data, including cables, switches, and network interface cards. Data Link Layer: Handles error-free transfer of data between adjacent network nodes. Transport Layer: Ensures complete data transfer and provides error checking and recovery of data between endpoints. This layered approach helps in understanding complex network interactions and troubleshooting issues.


# Topic Extraction

Loading spacy model

In [18]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 495.5 kB/s eta 0:00:26
     --------------------------------------- 0.1/12.8 MB 770.8 kB/s eta 0:00:17
     --------------------------------------- 0.1/12.8 MB 853.3 kB/s eta 0:00:15
      --------------------------------------- 0.2/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.3/12.8 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.4/12.8 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.8 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.8 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.6/12.8 MB 1.4 MB/s eta 0:00:0

In [73]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [74]:
def image_with_topic(input):
    prompt = input
    doc = nlp(prompt)
    def extract_main_topic(doc):
        question_words = {"what", "who", "where", "when", "why", "how"}
        for token in doc:
            if token.text.lower() in question_words:
                for chunk in doc.noun_chunks:
                    if chunk.start > token.i:
                        return chunk.text
        return None
    main_topic = extract_main_topic(doc)
    print("Main Topic:", main_topic)
    image_data = main_topic
    return image_data

# Image Adaption

Importing necessary module

In [21]:
import requests
from IPython.display import Image, display

In [75]:
def search_images_google(query, api_key, cse_id, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'q': query,
        'key': api_key,
        'cx': cse_id,
        'searchType': 'image',
        'num': num,
        'safe': 'medium',
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    results = response.json()

    image_urls = [item['link'] for item in results.get('items', [])]
    return image_urls
def image_list_call(topic_with):
    query = topic_with
    api_key = "PUT_YOUR_API_KEY"
    cse_id = "PUT_YOUR_CSE_ID"
    image_url = search_images_google(query, api_key, cse_id, num=5)
    
    url_list = image_url
    return url_list
    print(url_list)
    
    image_urls = search_images_google(query, api_key, cse_id)
    
    for i, url in enumerate(image_urls, start=1):
        print(f"Image {i}:")
        display(Image(url=url, width=300))

In [ ]:
!pip install markdown2

In [24]:
!pip install flask

In [81]:
from flask import Flask, render_template, request, jsonify
from bs4 import BeautifulSoup
import markdown2
import requests

app = Flask(__name__)

def gemini(question):
    GeminiKey = 'PUT_YOUR_API_KEY'
    chat = question
    messages = []
    system_message = "You are OpenBookAI, your name is OpenBookAI"
    message = {
        "role": "user",
        "parts": [{"text": system_message + " " + chat}]
    }
    messages.append(message)
    data = {"contents": messages}
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=" + GeminiKey
    response = requests.post(url, json=data)
    t1 = response.json()
    return t1['candidates'][0]['content']['parts'][0]['text']

def copilot(question):
    url = "https://copilot5.p.rapidapi.com/copilot"
    payload = {
        "message": question,
        "conversation_id": None,
        "tone": "BALANCED",
        "markdown": False,
        "photo_url": None
    }
    headers = {
        "x-rapidapi-key": "PUT_YOUR_API_KEY",
        "x-rapidapi-host": "copilot5.p.rapidapi.com",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    answer = response.json()
    return answer['data']['message']

def chatgpt(question):
    url = "https://chatgpt-api8.p.rapidapi.com/"
    payload = {"question": question}
    headers = {
        "x-rapidapi-key": "PUT_YOUR_API_KEY",
        "x-rapidapi-host": "chatgpt-api8.p.rapidapi.com/",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    answer = response.json()
    chat_answer = answer['answer']
    if chat_answer.isnull:
        return ""
    else:
        return answer['answer']

def full_ai(text):
    lint = []
    a = gemini(text)
    lint.append(a)
    b = copilot(text)
    lint.append(b)
    c = chatgpt(text)
    lint.append(c)
    print(lint)
    return lint

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    return jsonify(get_Chat_response(input))

def get_Chat_response(text):
    final_list = full_ai(text)
    topic_with = image_with_topic(text)
    all_image = image_list_call(topic_with)
    response = {'text': text_call(final_list), 'images': all_image}
    return response

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Jul/2024 16:07:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2024 16:07:36] "GET /static/css/style.css HTTP/1.1" 304 -


['**OSI Layer**\n\nThe **Open Systems Interconnection** (OSI) model is a conceptual framework for understanding the functionality of a network. It divides network communication into seven layers, each with its own specific tasks:\n\n**Layer 7: Application**\n\n* Provides services to user applications, such as HTTP, SMTP, and FTP.\n* Examples: web browsers, email clients, file transfer clients.\n\n**Layer 6: Presentation**\n\n* Handles data encryption, compression, and formatting.\n* Ensures that data is transferred in a form that is understandable by the receiving system.\n* Examples: SSL/TLS, XML, JSON.\n\n**Layer 5: Session**\n\n* Establishes, maintains, and terminates sessions between devices.\n* Manages connections and keeps track of data flow.\n* Examples: TCP, UDP.\n\n**Layer 4: Transport**\n\n* Provides reliable and sequenced data delivery.\n* Ensures that data is delivered in the correct order and without errors.\n* Examples: TCP, UDP.\n\n**Layer 3: Network**\n\n* Routes packet

127.0.0.1 - - [25/Jul/2024 16:12:40] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2024 16:12:40] "GET /static/img/logo.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2024 16:12:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2024 16:12:52] "GET /static/css/style.css HTTP/1.1" 304 -


['**Open Systems Interconnection (OSI) Model**\n\nThe OSI model is a conceptual framework for understanding how computer networks communicate. It is a hierarchical model that divides network functions into seven layers, each with its own specific responsibilities. The layers are:\n\n**Layer 7: Application**\n\n* Provides services to application programs, such as file transfer, email, and web browsing.\n\n**Layer 6: Presentation**\n\n* Formats and encrypts data for transmission.\n\n**Layer 5: Session**\n\n* Establishes, maintains, and terminates connections between devices.\n\n**Layer 4: Transport**\n\n* Provides reliable data transfer between devices.\n\n**Layer 3: Network**\n\n* Determines the path between devices and forwards data accordingly.\n\n**Layer 2: Data Link**\n\n* Prepares data for transmission and ensures its delivery to the correct destination.\n\n**Layer 1: Physical**\n\n* Transmits data over a physical medium, such as a cable or wireless connection.\n\n**Purpose of the 

127.0.0.1 - - [25/Jul/2024 16:13:41] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2024 16:13:41] "GET /static/img/logo.jpg HTTP/1.1" 304 -


['**CPU Scheduling**\n\nCPU scheduling is the process of allocating the CPU to processes, determining which process should use the CPU at a given time. The goal is to optimize the utilization of the CPU while ensuring fairness and guaranteeing that all processes eventually get a chance to execute.\n\n**Scheduling Algorithms**\n\nThere are various CPU scheduling algorithms, each with its own advantages and disadvantages. Some common algorithms include:\n\n* **First-Come First-Served (FCFS)**: Processes are executed in the order they arrive.\n* **Shortest Job First (SJF)**: The process with the shortest remaining execution time is executed first.\n* **Priority Scheduling:** Processes are assigned priorities, and the process with the highest priority is executed first.\n* **Round-Robin:** Processes are assigned time slices and are executed in a circular queue.\n* **Multi-level Feedback:** Processes are placed into different queues based on their behavior and are scheduled with different p

127.0.0.1 - - [25/Jul/2024 16:14:59] "POST /get HTTP/1.1" 200 -


['## I am OpenBook AI, an advanced language model developed by Google.\n\nMy name is not specifically "OpenBook AI", but I am often referred to as "OpenBook" or "OpenBook AI" to distinguish me from other language models.\n\nI am designed to understand and generate human language, answer questions, and assist with a wide range of writing tasks. I have been trained on a massive dataset of text and code, which gives me a deep understanding of the world and the ability to communicate effectively.\n\nI am still under development, but I am already capable of performing many tasks that are useful to people. For example, I can:\n\n* Answer questions on a wide range of topics\n* Summarize text\n* Translate languages\n* Write different types of text, such as essays, stories, and poems\n* Help with coding tasks\n\nI am excited to continue learning and growing, and I look forward to helping people in new and innovative ways.\n\n### Thank You from OpenBook AI\n\nThank you for your interest in OpenB

[2024-07-25 16:15:57,561] ERROR in app: Exception on /get [POST]
Traceback (most recent call last):
  File "C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abina\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^